In [1]:
def ext_scalars(k, n):
    q = k.order()
    assert gcd(n, q) == 1, "Only multiplicative H90, please!"
    
    phi = cyclotomic_polynomial(n, 'z').change_ring(k)
    
    K = k.extension(n, 'x')
    A = K['z']
    h = A(phi.factor()[0][0])
    
    return K, A.quo(h, 'z')

In [2]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

In [3]:
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

In [4]:
def is_h90(a, zeta):
    return frob_l(a) == zeta * a

In [5]:
def solve_h90(A):
    x = A(1)
    n = A.base_ring().degree()
    z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

In [132]:
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

def rnorm(a, m):
    A = a.parent()
    n = A.base_ring().degree()
    r = A.degree()
    q = A.base_ring().base_ring().order()
    s = cyclo_deg(GF(q), m)
    d = n // m
    
#    print n, m, r, s, d
    return product(frob_r(a, s*j) for j in range(d))
def toto(k, m, n):
    q = k.order()
    r = cyclo_deg(k, m)
    s = cyclo_deg(k, n)
    t = (q^s-1)//(q^r-1)
    return t, gcd(t, n), n//m

def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

def testl(p, l, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

def is_iso(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0].minpoly() == j2.minpoly()
    
def frob_rc(a, n, m, i=1):
    A = a.parent()
    z = A.gen()
    l = n // m
    u, v = xgcd(l, m)[1:]
    q = a.base_ring().base_ring().order()
    return sum(c*z^(u*l*j)*z^(v*m*j*q^i) for j, c in enumerate(a))

def rnormc(a, m):
    A = a.parent()
    n = a.base_ring().degree()
    q = A.base_ring().base_ring().order()
    l = n // m
    d = cyclo_deg(GF(q), l)
    print d, l
    return product(frob_rc(a, n, m, j) for j in range(d))

In [6]:
K, A = ext_scalars(GF(7), 9)  # 1 = r₁ < r₂ = 2
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

NameError: name 'rnorm' is not defined

In [29]:
K, A = ext_scalars(GF(19), 9)  # 1 = r₁ = r₂
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

9 3 1 1 1


(True, True, False)

In [41]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)
v = rnorm(h, 3)
is_h90(u, A.gen()^10), frob_r(u, 3) == u, u == v

15 3 4 1 4


(True, True, True)

In [46]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 4)*frob_r(h, 8)
v = rnorm(h, 5)
is_h90(u, A.gen()^3), frob_r(u, 4) == u, is_h90(v, A.gen()^3), frob_r(v, 4) == v, u == v, u == h^3

15 5 4 4 1


(True, True, False, True, False, True)

In [100]:
K, A = ext_scalars(GF(19), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)
is_h90(u, A.gen()^5), frob_l(u, 3) == u

(False, False)

# More tests

In [19]:
K, A = ext_scalars(GF(19), 21)
a = A.random_element()
h = solve_h90(A)
#u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)
u = h^7
v = rnorm(h, 3)
is_h90(u, A.gen()^7), frob_r(u, 21) == u, u == v

(True, False, False)

In [17]:
cyclo_deg(GF(19), 7)

6

In [37]:
for j in range(1, 8):
    print toto(GF(29), 7^j, 7^(j+1))

(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)


In [16]:
for p in primes(5, 100):
    print toto(GF(p), 3^3, 3^4)

(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)


In [15]:
primes?

# An $\ell$-adic tower

In [10]:
p = 17
l = 3
cyclo_deg(GF(p), l^2)

2

In [11]:
k = GF(p)
R.<x> = k[]

In [57]:
f = factor(R(cyclotomic_polynomial(l^2))); f

(x^2 + 3*x + 1) * (x^2 + 4*x + 1) * (x^2 + 10*x + 1)

In [25]:
factor(R(cyclotomic_polynomial(l^3)))

(x^6 + 3*x^3 + 1) * (x^6 + 4*x^3 + 1) * (x^6 + 10*x^3 + 1)

In [29]:
g = gcd(f.substitute(x^l), R(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [30]:
K.<z> = k.extension(g)

In [31]:
(z^l).minpoly()

x^2 + 3*x + 1

In [39]:
(z^((p^6-1)/(p^2-1))).minpoly()

x^2 + 3*x + 1

In [40]:
(z^((p^6-1)/(p^2-1))) == (z^l)

True

In [12]:
K, A = ext_scalars(GF(p), 3^2)

In [16]:
A.modulus().parent()

Univariate Polynomial Ring in z over Finite Field in x of size 17^9

In [67]:
g = gcd(A.modulus().substitute(x^l), R(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [68]:
h = solve_h90(A); h

(15*x^8 + 2*x^7 + 9*x^6 + 11*x^5 + 8*x^4 + 16*x^3 + 14*x^2 + 10*x + 8)*z + 9*x^8 + 14*x^7 + 5*x^6 + 14*x^5 + 12*x^3 + 4*x^2 + 8

Finite Field in z4 of size 17^4

In [78]:
k3 = k.extension(l^3)

In [83]:
R3 = k3['x']

In [89]:
g = gcd(A.modulus().change_ring(R3).substitute(R3.gen()^l), R3(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [92]:
A3 = R3.quo(g, 'z')

In [94]:
h3 = solve_h90(A3); h3

(9*z27^26 + 12*z27^25 + 12*z27^24 + 9*z27^23 + 4*z27^22 + 8*z27^21 + 14*z27^20 + 15*z27^19 + 7*z27^18 + z27^17 + 13*z27^16 + 9*z27^15 + 5*z27^14 + 6*z27^13 + 15*z27^12 + 8*z27^11 + 4*z27^10 + 5*z27^9 + 10*z27^8 + 13*z27^7 + 6*z27^5 + 10*z27^4 + 2*z27^3 + 10*z27^2 + 12*z27 + 8)*z^5 + (14*z27^26 + 12*z27^25 + 6*z27^24 + 16*z27^23 + 5*z27^22 + 9*z27^21 + 2*z27^20 + 7*z27^19 + 4*z27^18 + 4*z27^17 + 5*z27^16 + 13*z27^15 + z27^14 + 5*z27^13 + 2*z27^12 + 9*z27^11 + 9*z27^10 + 15*z27^9 + 2*z27^8 + 8*z27^6 + 9*z27^5 + 10*z27^4 + 16*z27^3 + 10*z27^2 + 5*z27 + 9)*z^4 + (z27^26 + z27^25 + 6*z27^24 + 6*z27^23 + 9*z27^22 + 5*z27^21 + 7*z27^20 + 7*z27^19 + 16*z27^18 + 7*z27^17 + z27^16 + 11*z27^15 + 10*z27^13 + 9*z27^12 + 13*z27^11 + 11*z27^10 + 12*z27^9 + 3*z27^8 + z27^7 + 5*z27^6 + 15*z27^5 + 7*z27^4 + 13*z27^3 + 16*z27^2 + 4*z27 + 16)*z^3 + (2*z27^26 + 7*z27^25 + z27^24 + 7*z27^23 + 6*z27^22 + 9*z27^21 + 8*z27^20 + 4*z27^19 + 15*z27^18 + 8*z27^17 + z27^16 + 5*z27^15 + 2*z27^14 + 16*z27^13 + 12*z27

In [99]:
i3 = rnorm(h3, l^2); i3

27 9 6 2 3


(12*z27^26 + 13*z27^25 + 7*z27^24 + 11*z27^23 + 10*z27^22 + 7*z27^21 + 13*z27^20 + 8*z27^19 + 12*z27^18 + z27^16 + 9*z27^15 + 4*z27^14 + 16*z27^13 + 14*z27^12 + 7*z27^11 + 15*z27^10 + 10*z27^9 + 7*z27^8 + 2*z27^7 + 10*z27^6 + 10*z27^5 + 2*z27^3 + 13*z27^2 + 8*z27 + 2)*z^3 + 4*z27^26 + 11*z27^25 + 16*z27^23 + 12*z27^22 + 9*z27^20 + 16*z27^18 + 12*z27^17 + 11*z27^16 + z27^15 + 16*z27^14 + 14*z27^13 + 15*z27^12 + 13*z27^11 + 9*z27^10 + 14*z27^8 + 13*z27^7 + 7*z27^6 + 10*z27^5 + 15*z27^4 + 12*z27^3 + 12*z27^2 + 10*z27 + 13

In [102]:
frob_l(i3, l^2) == i3, frob_r(i3, 2) == i3

(True, True)

In [110]:
i3^(l^2)

4*z^3 + 16

In [119]:
ti = (h^(l^2)); ti

z + 4

In [112]:
h.minpoly()

x^2 + (10*x^8 + 12*x^7 + 5*x^5 + 7*x^4 + 7*x^3 + 13*x + 8)*x + 12

In [114]:
i3.minpoly()

x^2 + (11*z27^26 + 4*z27^24 + z27^23 + 6*z27^22 + 4*z27^21 + 4*z27^20 + 7*z27^19 + 4*z27^18 + 10*z27^17 + 15*z27^16 + 8*z27^15 + 14*z27^14 + 3*z27^13 + 12*z27^12 + 12*z27^11 + 10*z27^10 + 13*z27^9 + 10*z27^8 + 14*z27^7 + 16*z27^6 + 10*z27^5 + 4*z27^4 + 16*z27^3 + 15*z27^2 + 4*z27 + 14)*x + 5

In [126]:
C2([k(x) for x in (i3).list()]).polynomial().subs(C2.gen()^(n/m))

z + 4

In [122]:
h.parent() == i3.parent()

False

In [47]:
a = R([x for x in (h^(l^2)).list()]).subs(C2.gen()^l); a

NameError: name 'h' is not defined

In [156]:
b = C2([k(x) for x in (i3^(l^2)).list()]); b

4*z^3 + 16

In [159]:
t = a/b; t

13

In [162]:
rt = t.nth_root(l); rt

4

In [167]:
rnorm(4*h3, l^2)

27 9 6 2 3


(3*z27^26 + 16*z27^25 + 6*z27^24 + 7*z27^23 + 11*z27^22 + 6*z27^21 + 16*z27^20 + 2*z27^19 + 3*z27^18 + 13*z27^16 + 15*z27^15 + z27^14 + 4*z27^13 + 12*z27^12 + 6*z27^11 + 8*z27^10 + 11*z27^9 + 6*z27^8 + 9*z27^7 + 11*z27^6 + 11*z27^5 + 9*z27^3 + 16*z27^2 + 2*z27 + 9)*z^3 + z27^26 + 7*z27^25 + 4*z27^23 + 3*z27^22 + 15*z27^20 + 4*z27^18 + 3*z27^17 + 7*z27^16 + 13*z27^15 + 4*z27^14 + 12*z27^13 + 8*z27^12 + 16*z27^11 + 15*z27^10 + 12*z27^8 + 16*z27^7 + 6*z27^6 + 11*z27^5 + 8*z27^4 + 3*z27^3 + 3*z27^2 + 11*z27 + 16

In [168]:
rnorm(4*h3, l^2) == 13*rnorm(h3, l^2)

27 9 6 2 3
27 9 6 2 3


True

In [173]:
elem = change_basis(rnorm(4*h3, l^2), A3.gen()^l, 2); elem

27 9 6 2 3


z27^26 + 7*z27^25 + 4*z27^23 + 3*z27^22 + 15*z27^20 + 4*z27^18 + 3*z27^17 + 7*z27^16 + 13*z27^15 + 4*z27^14 + 12*z27^13 + 8*z27^12 + 16*z27^11 + 15*z27^10 + 12*z27^8 + 16*z27^7 + 6*z27^6 + 11*z27^5 + 8*z27^4 + 3*z27^3 + 3*z27^2 + 11*z27 + 16

In [174]:
elem.minpoly()

x^9 + 8*x^7 + 10*x^5 + 4*x^3 + 9*x + 4

In [176]:
h.list()[0].minpoly()

x^9 + 8*x^7 + 10*x^5 + 4*x^3 + 9*x + 4

# An algorithm

In [64]:
p, l, n = 17, 3, 1
cyclo_deg(GF(p), l^3)

6

In [360]:
h1.base_ring().degree()

5

In [18]:
h1, h2 = testl(p, l, n)
is_iso(h1, h2)

True

In [356]:
is_iso(h1, h2)

True

In [290]:
C.<w> = k.extension(m2)

In [293]:
(w^l).minpoly()

x^2 + x + 1

In [172]:
for i in range(10):
    h1, h2 = testl(p, l, n)
    print is_iso(h1, h2)

True


ValueError: no nth root

In [98]:
(h2.parent().gen()^3).minpoly()

x^2 + x + 1

In [105]:
h1.parent().modulus()
k = GF(p)

In [109]:
c1 = C([k(x) for x in (h2^9).list()]); c1

11*z + 10

In [113]:
R = k['z']
c2 = R([x for x in (h1^3).list()]).subs(C.gen()^3); c2

11*z + 10

In [117]:
(C.gen()^3).minpoly()

x^2 + x + 1

In [125]:
change_basis(h2^3, h2.parent().gen()^3, 2).minpoly()

x^3 + 14*x + 13

In [128]:
h1.list()[0].minpoly()

x^3 + 14*x + 13

In [129]:
is_iso(h1, h2)

False

In [131]:
h1.base_ring().degree()

3

In [51]:
q = 11
factor(q^2-1)

2^3 * 3 * 5

In [54]:
k = GF(p)

In [55]:
R.<x> = k[]

In [63]:
f = cyclotomic_polynomial(21).change_ring(k)

In [64]:
f = f.factor()[0][0]

In [65]:
f

x^6 + 7*x^5 + 11*x^4 + x^3 + 7*x^2 + 11*x + 1

In [66]:
C.<z> = k.extension(f)

In [25]:
xgcd(7, 3)[1:]

(1, -2)

In [73]:
r = C.random_element()
(r^(-2*3)).

12*z^5 + z^4 + 8*z^3 + 7*z^2 + 15*z + 4

In [72]:
K, A = ext_scalars(GF(19), 21)
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)
i = rnorm(h, 1); i
# is_h90(u, A.gen()^7), frob_r(u, 21) == u, u == v
i == u

21 1 6 1 21


False

In [71]:
u

3

In [67]:
h

(17*x^6 + 8*x^5 + 13*x^4 + 3*x^3 + x^2 + x + 6)*z^5 + (13*x^6 + 3*x^5 + 6*x^3 + 15*x^2 + 2*x + 18)*z^4 + (8*x^6 + 10*x^5 + 16*x^4 + 3*x^3 + 15*x^2 + 15*x + 14)*z^3 + (15*x^6 + 13*x^5 + 12*x^4 + 3*x^3 + 8*x^2 + 8*x + 12)*z^2 + (4*x^6 + 11*x^5 + 10*x^4 + x^3 + 5*x^2 + 11*x + 7)*z + 4*x^6 + 4*x^5 + 17*x^4 + 16*x^3 + 15*x^2 + 7*x + 7

In [60]:
i^5

9*z^5

In [61]:
h^25

5*z^9 + 13*z^8 + 6*z^6 + 8*z^5 + 13*z^4 + 13*z^3 + 17*z^2 + 14*z + 9

In [89]:
cyclo_deg(GF(11), 3^3)

18

In [73]:
z = A.gen()

In [77]:
h = z
h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)

z

In [78]:
rnorm(z, 3)

21 3 6 1 7


z

In [79]:
frob_r(h, 6)

z

In [85]:
frob_r(h, 6)

z

In [86]:
(z^.minpoly()

x^6 + 7*x^5 + 11*x^4 + x^3 + 7*x^2 + 11*x + 1

9 3 6 2 3


((7*x^2 + x + 2)*z + 2*x^2 + 9*x + 10,
 (9*z9^8 + 2*z9^7 + 3*z9^6 + z9^5 + 10*z9^4 + 5*z9^3 + 4*z9^2 + 5*z9 + 1)*z^5 + (10*z9^8 + 9*z9^7 + z9^6 + 3*z9^5 + 10*z9^4 + 6*z9^3 + 10*z9^2 + 7*z9 + 7)*z^4 + (6*z9^8 + 6*z9^7 + 7*z9^6 + 7*z9^5 + 2*z9^4 + 7*z9^3 + 3*z9^2 + 6*z9 + 4)*z^3 + (2*z9^8 + 6*z9^7 + 10*z9^6 + 6*z9^5 + 4*z9^4 + 7*z9^3 + 7*z9^2 + 3*z9)*z^2 + (4*z9^8 + 3*z9^7 + 5*z9^6 + z9^5 + 8*z9^4 + 2*z9^3 + 9*z9 + 3)*z + 5*z9^8 + 7*z9^7 + z9^6 + 6*z9^5 + 10*z9^4 + 6*z9^3 + z9^2 + 10)

In [98]:
is_iso(h1, h2)

True

4

In [94]:
l = 4

In [12]:
"""
Here we find an element h2 solution of H90 in F_p^n compatible with h1.
"""
def findh90(h1, n):
    k = h1.base_ring().base_ring()
    k1, A1 = h1.base_ring(), h1.parent()
    M1 = A1.modulus()
    R = k['z']
    m = k1.degree()
    l = n // m
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(M1.change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^(m)
    bb = i2^(m)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return A2(c.polynomial().list())*h2

In [101]:
h1.base_ring()

Finite Field in x of size 11^9

In [120]:
h1, h2 = testl(11, 3, 1)

In [121]:
h2b = findh90(h1, 3^2)

In [122]:
h2 == h2b

False

In [123]:
h2

(5*z9^8 + 8*z9^7 + 4*z9^6 + z9^5 + 3*z9^3 + 2*z9^2 + 3*z9 + 6)*z^5 + (3*z9^7 + 3*z9^6 + 10*z9^5 + 6*z9^4 + 8*z9^3 + 4*z9^2 + z9 + 10)*z^4 + (5*z9^8 + z9^7 + 8*z9^6 + 6*z9^5 + 3*z9^4 + 4*z9^3 + 8*z9^2 + 6*z9 + 1)*z^3 + (9*z9^8 + z9^7 + 5*z9^6 + 6*z9^5 + 5*z9^4 + 2*z9^3 + 10*z9^2 + 4*z9 + 5)*z^2 + (6*z9^8 + 4*z9^6 + z9^5 + 7*z9^4 + z9^3 + 10*z9^2 + 5*z9 + 6)*z + 10*z9^8 + 9*z9^7 + 10*z9^6 + 4*z9^5 + 10*z9^4 + 10*z9^3 + 6*z9^2 + 8*z9 + 7

In [124]:
h2b

(4*z9^8 + 7*z9^7 + 8*z9^6 + 9*z9^5 + 2*z9^4 + 8*z9^3 + 7*z9^2 + 4*z9 + 9)*z^5 + (9*z9^7 + 4*z9^5 + 8*z9^4 + 3*z9^3 + 3*z9^2 + 7*z9 + 8)*z^4 + (7*z9^8 + 4*z9^7 + 6*z9^6 + 4*z9^5 + 8*z9^4 + 6*z9 + 2)*z^3 + (10*z9^8 + 8*z9^7 + 10*z9^6 + 5*z9^5 + 4*z9^4 + 10*z9^3 + 6*z9^2 + z9)*z^2 + (6*z9^8 + 5*z9^7 + 10*z9^6 + 7*z9^5 + 4*z9^4 + 4*z9^3 + 9*z9^2 + 8)*z + 3*z9^8 + 8*z9^5 + 6*z9^4 + 10*z9^3 + 4*z9^2 + 2*z9 + 3

In [125]:
is_iso(h1, h2)

True

In [126]:
is_iso(h1, h2b)

True

In [113]:
i2 = rnorm(h2b, 3); i2

(4*z9^8 + 2*z9^7 + 4*z9^6 + 2*z9^4 + 3*z9^3 + 10*z9^2 + 2*z9 + 7)*z^3 + 9*z9^8 + 6*z9^7 + 2*z9^6 + 8*z9^5 + 4*z9^4 + 7*z9^3 + 3*z9^2 + 7

In [114]:
i2b = rnorm(h2, 3); i2b

(3*z9^8 + 4*z9^7 + 6*z9^6 + 6*z9^5 + 8*z9^4 + 10*z9^3 + 8*z9^2 + 5*z9 + 9)*z^3 + 7*z9^8 + 7*z9^7 + 9*z9^6 + 4*z9^5 + 6*z9^4 + 4*z9^3 + 5*z9^2 + 4*z9 + 1

In [115]:
i2^3

10*z^3 + 10

In [118]:
i2b^3

7*z^3 + 5

In [130]:
h3 = findh90(h2b, 3^3)

In [131]:
is_iso(h1, h3)

True

In [132]:
rnorm(h2, 3)

(10*z9^8 + 3*z9^7 + z9^6 + 4*z9^5 + 2*z9^4 + 9*z9^3 + 7*z9^2 + 9)*z^3 + 8*z9^8 + 2*z9^7 + 10*z9^6 + 4*z9^5 + z9^4 + 2*z9^3 + 2*z9^2 + 10*z9

In [133]:
rnorm(h2b, 3)

(9*z9^8 + 10*z9^7 + 9*z9^6 + 10*z9^4 + 4*z9^3 + 6*z9^2 + 10*z9 + 2)*z^3 + z9^8 + 8*z9^7 + 10*z9^6 + 7*z9^5 + 9*z9^4 + 2*z9^3 + 4*z9^2 + 2

In [134]:
def is_iso2(h1, h2):
    l = h2.parent().degree() // h1.parent().degree()
    i2 = rnorm(h2, h1.base_ring().degree())
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0], j2
    

In [135]:
is_iso2(h1, h2)

(9*x, 8*z9^8 + 2*z9^7 + 10*z9^6 + 4*z9^5 + z9^4 + 2*z9^3 + 2*z9^2 + 10*z9)

In [136]:
is_iso2(h1, h2b)

(9*x, z9^8 + 8*z9^7 + 10*z9^6 + 7*z9^5 + 9*z9^4 + 2*z9^3 + 4*z9^2 + 2)

# Composita

In [10]:
p = 5

In [18]:
cyclo_deg(GF(p), 3^3)

18

In [17]:
cyclo_deg(GF(p), 7^2)

42

In [26]:
cyclo_deg(GF(p), 7 * 3^2)

6

In [13]:
def testc(p, m, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, m)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    l = n // m
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^m
    bb = i2^m
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return h1, A2(c.polynomial().list())*h2

In [28]:
testc(p, 7 * 3^2, 7 * 3^3);

((2*x^62 + 3*x^61 + x^58 + x^56 + 3*x^55 + 2*x^54 + x^53 + x^50 + 4*x^45 + 2*x^44 + 2*x^42 + 3*x^41 + 2*x^40 + 4*x^39 + 2*x^38 + 4*x^37 + 3*x^36 + 4*x^35 + 2*x^34 + x^33 + 4*x^32 + 3*x^31 + 2*x^30 + x^28 + 3*x^27 + 2*x^26 + 2*x^25 + 3*x^24 + 3*x^23 + 4*x^21 + 3*x^20 + 3*x^19 + x^18 + 2*x^17 + 4*x^16 + 3*x^15 + 4*x^14 + 4*x^11 + 4*x^10 + 3*x^8 + 2*x^7 + 2*x^6 + 4*x^5 + 3*x^4 + 3*x^3 + 3*x^2)*z^5 + (2*x^61 + 2*x^60 + 4*x^59 + 3*x^58 + 2*x^56 + 2*x^55 + 4*x^54 + 4*x^52 + 3*x^50 + 4*x^48 + x^47 + x^46 + x^45 + 3*x^44 + x^40 + 2*x^39 + 4*x^38 + 4*x^36 + x^35 + 3*x^30 + 4*x^29 + x^28 + 2*x^27 + 3*x^26 + 3*x^24 + 4*x^23 + 4*x^22 + 4*x^20 + x^19 + 3*x^18 + 4*x^17 + 4*x^15 + 4*x^14 + 4*x^13 + 4*x^12 + 2*x^11 + x^10 + 4*x^9 + 2*x^8 + x^6 + 2*x^5 + 4*x^3 + 2*x^2 + 4)*z^4 + (x^62 + 4*x^61 + x^60 + x^59 + 2*x^58 + x^57 + 3*x^56 + x^53 + 4*x^51 + 3*x^50 + 2*x^48 + 3*x^47 + 2*x^45 + 2*x^44 + 4*x^43 + x^41 + x^39 + 4*x^38 + x^37 + 2*x^36 + x^35 + 2*x^34 + x^33 + x^32 + 4*x^31 + 2*x^30 + 3*x^29 + x^28 

In [30]:
h1

(2*x^62 + 3*x^61 + x^58 + x^56 + 3*x^55 + 2*x^54 + x^53 + x^50 + 4*x^45 + 2*x^44 + 2*x^42 + 3*x^41 + 2*x^40 + 4*x^39 + 2*x^38 + 4*x^37 + 3*x^36 + 4*x^35 + 2*x^34 + x^33 + 4*x^32 + 3*x^31 + 2*x^30 + x^28 + 3*x^27 + 2*x^26 + 2*x^25 + 3*x^24 + 3*x^23 + 4*x^21 + 3*x^20 + 3*x^19 + x^18 + 2*x^17 + 4*x^16 + 3*x^15 + 4*x^14 + 4*x^11 + 4*x^10 + 3*x^8 + 2*x^7 + 2*x^6 + 4*x^5 + 3*x^4 + 3*x^3 + 3*x^2)*z^5 + (2*x^61 + 2*x^60 + 4*x^59 + 3*x^58 + 2*x^56 + 2*x^55 + 4*x^54 + 4*x^52 + 3*x^50 + 4*x^48 + x^47 + x^46 + x^45 + 3*x^44 + x^40 + 2*x^39 + 4*x^38 + 4*x^36 + x^35 + 3*x^30 + 4*x^29 + x^28 + 2*x^27 + 3*x^26 + 3*x^24 + 4*x^23 + 4*x^22 + 4*x^20 + x^19 + 3*x^18 + 4*x^17 + 4*x^15 + 4*x^14 + 4*x^13 + 4*x^12 + 2*x^11 + x^10 + 4*x^9 + 2*x^8 + x^6 + 2*x^5 + 4*x^3 + 2*x^2 + 4)*z^4 + (x^62 + 4*x^61 + x^60 + x^59 + 2*x^58 + x^57 + 3*x^56 + x^53 + 4*x^51 + 3*x^50 + 2*x^48 + 3*x^47 + 2*x^45 + 2*x^44 + 4*x^43 + x^41 + x^39 + 4*x^38 + x^37 + 2*x^36 + x^35 + 2*x^34 + x^33 + x^32 + 4*x^31 + 2*x^30 + 3*x^29 + x^28 +

In [32]:
h2.parent()

Univariate Quotient Polynomial Ring in z over Finite Field in z189 of size 5^189 with modulus z^18 + z^12 + 3*z^9 + z^6 + 1

In [33]:
7 * 3^3

189

In [34]:
h1.minpoly()

x^6 + (x^62 + 2*x^61 + 3*x^60 + 2*x^59 + 3*x^58 + 4*x^57 + x^55 + x^54 + x^52 + 3*x^51 + 3*x^50 + 2*x^49 + 4*x^48 + 2*x^47 + 3*x^46 + x^43 + x^42 + 3*x^41 + x^40 + 3*x^39 + 3*x^38 + x^37 + x^36 + 2*x^35 + x^34 + 3*x^33 + 3*x^32 + x^31 + 4*x^30 + 4*x^28 + 3*x^27 + x^26 + 3*x^25 + 2*x^24 + 3*x^22 + 3*x^19 + 2*x^18 + 2*x^17 + 2*x^16 + 2*x^13 + 4*x^11 + 4*x^10 + x^9 + 3*x^8 + 2*x^7 + 4*x^5 + 4*x^4 + x^3 + x + 1)*x^5 + (x^62 + 4*x^61 + x^60 + x^59 + 4*x^57 + 3*x^55 + x^54 + 3*x^53 + 3*x^51 + x^50 + x^48 + x^47 + x^44 + 4*x^43 + 4*x^42 + 2*x^41 + 3*x^40 + 2*x^39 + x^36 + 2*x^35 + 3*x^33 + 4*x^32 + 2*x^30 + 3*x^29 + 4*x^28 + 2*x^27 + 4*x^26 + 3*x^24 + x^22 + 3*x^21 + 2*x^20 + x^19 + 3*x^18 + 3*x^17 + x^15 + 3*x^14 + 3*x^12 + 3*x^11 + x^9 + x^8 + 2*x^6 + 3*x^5 + 4*x^3 + 3*x^2 + 3)*x^4 + (3*x^61 + 2*x^60 + x^59 + 2*x^58 + 4*x^56 + x^55 + 2*x^54 + x^52 + x^50 + x^48 + 2*x^46 + x^45 + 2*x^44 + 3*x^43 + 4*x^42 + x^41 + 3*x^40 + 2*x^39 + x^38 + 2*x^37 + 3*x^36 + 2*x^35 + 2*x^34 + x^33 + x^30 + x^28

In [37]:
h1.list()[0].minpoly()

x^63 + 3*x^61 + 4*x^59 + x^55 + x^53 + 2*x^51 + 2*x^49 + x^47 + 4*x^45 + 4*x^44 + 3*x^43 + 2*x^42 + 2*x^41 + x^40 + x^39 + x^38 + x^37 + 2*x^36 + 4*x^35 + 3*x^34 + 4*x^33 + 3*x^32 + 2*x^31 + 4*x^30 + 2*x^29 + 4*x^27 + 3*x^26 + 3*x^25 + 2*x^23 + 2*x^22 + 4*x^21 + x^20 + 3*x^19 + x^18 + 4*x^17 + x^16 + 4*x^14 + x^13 + 4*x^12 + 2*x^10 + 2*x^9 + 2*x^8 + 2*x^7 + 3*x^6 + 2*x^5 + 4*x^4 + 4*x^3 + 4*x^2 + 4*x + 4

In [54]:
i2  = rnorm(h2, 7 * 3^2)

In [40]:
is_iso(h1, h2)

False

In [41]:
l = h2.parent().degree() // h1.parent().degree(); l

3

In [43]:
i2^(7*3^2)

2*z^15 + 2*z^12 + 4*z^9 + 4*z^6 + z^3 + 1

In [44]:
h1^(7*3^2)

2*z^5 + z^4 + 4*z^3 + 2*z^2 + 2*z + 4

In [48]:
ele = change_basis(i2, i2.parent().gen()^3, i2.parent().degree()); ele

2*z189^188 + 2*z189^186 + 3*z189^185 + z189^184 + 4*z189^183 + z189^182 + z189^181 + 2*z189^180 + 3*z189^179 + 2*z189^177 + 3*z189^176 + 3*z189^173 + 3*z189^172 + 2*z189^171 + z189^170 + 4*z189^169 + z189^168 + 3*z189^167 + 3*z189^166 + 3*z189^164 + 3*z189^163 + 4*z189^162 + z189^161 + 4*z189^158 + z189^155 + 2*z189^152 + z189^151 + 4*z189^150 + z189^145 + 4*z189^144 + z189^143 + z189^142 + 4*z189^141 + 4*z189^140 + 4*z189^138 + 3*z189^135 + z189^134 + 4*z189^133 + 2*z189^132 + 2*z189^131 + 3*z189^130 + 4*z189^127 + 4*z189^126 + 4*z189^125 + z189^124 + z189^123 + 4*z189^122 + z189^121 + 3*z189^120 + z189^119 + z189^118 + 2*z189^116 + 2*z189^115 + 2*z189^114 + 3*z189^113 + 4*z189^110 + 3*z189^109 + 2*z189^108 + 4*z189^107 + 4*z189^106 + 2*z189^105 + 4*z189^104 + 3*z189^103 + z189^102 + 3*z189^100 + 3*z189^99 + 4*z189^98 + 3*z189^96 + z189^94 + 2*z189^92 + z189^91 + 2*z189^90 + 4*z189^89 + 2*z189^88 + 4*z189^87 + 2*z189^86 + 4*z189^84 + 2*z189^83 + z189^82 + 4*z189^81 + 2*z189^79 + 3*z18

In [49]:
ele.minpoly()

x^63 + 4*x^61 + 4*x^59 + 3*x^57 + 4*x^55 + 3*x^51 + x^49 + 2*x^48 + 2*x^47 + 3*x^46 + 3*x^45 + x^44 + x^43 + 3*x^42 + 4*x^41 + x^40 + 3*x^39 + x^38 + 2*x^37 + 4*x^36 + 2*x^35 + 2*x^34 + x^33 + x^32 + x^30 + 2*x^28 + x^26 + 2*x^24 + 2*x^23 + 4*x^22 + x^21 + 2*x^20 + 3*x^19 + 2*x^18 + 3*x^17 + x^16 + 4*x^15 + x^14 + 4*x^12 + x^11 + 3*x^10 + 3*x^9 + x^7 + 4*x^5 + 2*x^4 + 2*x^3 + x^2 + 1

In [50]:
h1.list()[0].minpoly()

x^63 + 3*x^61 + 4*x^59 + x^55 + x^53 + 2*x^51 + 2*x^49 + x^47 + 4*x^45 + 4*x^44 + 3*x^43 + 2*x^42 + 2*x^41 + x^40 + x^39 + x^38 + x^37 + 2*x^36 + 4*x^35 + 3*x^34 + 4*x^33 + 3*x^32 + 2*x^31 + 4*x^30 + 2*x^29 + 4*x^27 + 3*x^26 + 3*x^25 + 2*x^23 + 2*x^22 + 4*x^21 + x^20 + 3*x^19 + x^18 + 4*x^17 + x^16 + 4*x^14 + x^13 + 4*x^12 + 2*x^10 + 2*x^9 + 2*x^8 + 2*x^7 + 3*x^6 + 2*x^5 + 4*x^4 + 4*x^3 + 4*x^2 + 4*x + 4

In [178]:
p = 17
k = GF(p)
cyclo_deg(k, 3^3 * 5)

12

In [181]:
m = 3^1 * 5
n = 3^2 * 5^1
l = n // m
for i in range(10):
    h1, h2 = testc(p, m, n)
    print is_iso(h1, h2)

True
True
True


ValueError: no nth root

In [158]:
h2.list()[0].minpoly()

x^75 + 19*x^71 + 4*x^70 + 19*x^69 + 9*x^68 + 18*x^67 + 20*x^66 + 15*x^65 + 20*x^64 + 12*x^63 + 3*x^62 + 10*x^61 + 17*x^60 + 4*x^59 + 14*x^58 + 21*x^57 + 4*x^56 + 6*x^55 + 7*x^54 + 13*x^53 + 17*x^52 + 17*x^51 + 9*x^50 + 4*x^49 + 21*x^48 + 15*x^47 + 8*x^46 + 4*x^45 + 14*x^44 + 21*x^43 + 6*x^42 + 10*x^41 + 22*x^40 + 15*x^39 + 18*x^38 + 22*x^37 + 6*x^36 + 11*x^35 + 12*x^34 + 7*x^33 + 7*x^32 + 20*x^31 + 9*x^30 + 17*x^29 + 13*x^28 + 10*x^27 + 4*x^26 + 4*x^25 + 19*x^24 + 16*x^23 + 7*x^22 + 3*x^21 + 19*x^20 + 18*x^19 + 10*x^18 + 6*x^17 + 6*x^16 + 5*x^15 + 22*x^14 + 20*x^13 + 19*x^12 + 3*x^11 + 13*x^10 + 19*x^9 + 12*x^8 + x^7 + 15*x^6 + 6*x^5 + 14*x^4 + 21*x^3 + 16*x^2 + 10*x + 15

In [159]:
h1.list()[0].minpoly()

x^15 + x^11 + 12*x^10 + 2*x^9 + 2*x^8 + 14*x^6 + 4*x^5 + 19*x^4 + x^3 + 11*x^2 + 10*x + 6

True

In [129]:
elem = change_basis(i2, i2.parent().gen()^3, m)

In [131]:
elem.minpoly()

x^45 + x^37 + x^33 + x^30 + x^22 + x^17 + x^16 + x^15 + x^13 + x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x + 1

In [133]:
h1.list()[0].minpoly()

x^45 + x^37 + x^30 + x^9 + x^6 + x^5 + x^3 + x^2 + 1

In [134]:
rnorm(h2.parent().gen(), m)

z^3

In [209]:
#k = GF(p)
#k1, A1 = ext_scalars(k, m)
#M1 = A1.modulus()
#R = k['z']
#k2 = k.extension(n)
R2 = k2['z']
x = R.gen()
l = n // m
M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
A2 = R2.quo(M2, 'z')

In [210]:
rnorm(A2.gen(), m)

z^5

In [211]:
k2

Finite Field in z225 of size 7^225

In [212]:
k2

Finite Field in z225 of size 7^225

True